In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time
from scipy.stats import rankdata
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.datasets import make_sparse_coded_signal
from scipy.stats import bernoulli

In [2]:
def GenerateData(N, M):
    a = np.zeros(M)
    b = np.zeros(M)
    m = np.zeros(M)
    n = np.zeros(M)
    
    for i in range(M-10):
        m[i] = np.random.binomial(N, 1/3)
        n[i] = np.random.binomial(N-m[i], 1/2)
        a[i] = np.random.binomial(m[i], 1/2)
        b[i] = np.random.binomial(n[i], 1/2)
    
    for i in range(M-10,M):
        m[i] = np.random.binomial(N, 1/3)
        n[i] = np.random.binomial(N-m[i], 1/2)
        a[i] = np.random.binomial(m[i], 3/4)
        b[i] = np.random.binomial(n[i], 3/4)
    
    stats = np.zeros(M)
    order = np.zeros(M)
    
    for i in range(M):
        order[i] = i
        if m[i] == 0:
            f = 0
        else:
            f = (2*a[i]-m[i])**2/m[i]
        if n[i] == 0:
            s = 0
        else:
            s = (2*b[i]-n[i])**2/n[i]
        if m[i] + n[i] == N:
            t = 0
        else:
            t = (2*a[i]+2*b[i]-m[i]-n[i])**2/(N-m[i]-n[i])
        
        stats[i] = f + s + t
    
    x = np.argsort(stats)
    #for i in range(10):
    #    print(x[M-1-i])
    
    stats = sorted(stats, reverse=True)
    #print(sorted_stats)
    
    return stats

In [3]:
def HaarMatrix(N):
    A = np.zeros((N,N))
    for j in range(0,N):
        A[0,j] = 1/np.sqrt(N)
    for i in range(1,N):
        for j in range(0,N):
            if N*(i-2**(np.floor(math.log2(i))))/(2**(np.floor(math.log2(i)))) <= j and j < N*(i-2**(np.floor(math.log2(i))) + 1/2)/(2**(np.floor(math.log2(i)))):
                A[i,j] = 2**(np.floor(math.log2(i))/2)/np.sqrt(N)
            elif N*(i-2**(np.floor(math.log2(i))) + 1/2)/(2**(np.floor(math.log2(i)))) <= j and j < N*(i-2**(np.floor(math.log2(i))) + 1)/(2**(np.floor(math.log2(i)))):
                A[i,j] = -2**(np.floor(math.log2(i))/2)/np.sqrt(N)
    return A

In [4]:
def BernoulliMatrix(K,N):
    phi = np.zeros((K,N))
    
    for i in range(K):
        for j in range(N):
            phi[i][j] = bernoulli.rvs(0.5, size=1)

    return (phi - 1/2) / (np.sqrt(K) / 2)

In [5]:
def OMP(A, y, N, K):
    x = np.zeros(N)
    S = np.zeros(N, dtype = np.uint8)
    r = y
    rr = np.dot(r,r)
    
    for i in range(K):
        err = rr - np.dot(A[:,S == 0].T, r) ** 2
        ndx = np.where(S == 0)[0]
        for j in range(K):
            S[j] = 1
        
        As = A[:, S == 1]
        pinv = np.linalg.pinv(np.dot(As, As.T))
        x[S==1] = np.dot(As.T, np.dot(pinv, y))
        
        r = y - np.dot(A, x)
        rr = np.dot(r,r)
    
    return x

In [6]:
def determineS(n,d,psi,eta):
    xx = np.dot(psi.T, d)
    
    s = 0
    for i in range(n):
        if xx[i] > eta:
            s += 1
    
    return s

In [7]:
def CompressiveMechanism(g, deltag, K, epsilon,eta):
    m = np.size(g)
    n = int(2**(np.ceil(math.log2(m))))
    
    #print(n)
    
    d = np.zeros(n)
    for i in range(m):
        d[i] = g[i]
    
    psi = HaarMatrix(n)
    
    s = determineS(n,d,psi,eta)
    k = int(np.floor(s*math.log(n/s)))
    
    phi = BernoulliMatrix(k,n)
    A = np.dot(phi, psi)
    
    y = np.dot(phi, d)
    pri_y = y + np.random.laplace(loc = 0.0, scale = 2*K*deltag*np.sqrt(k)/epsilon, size=k)
    
    pri_x = OMP(A, pri_y, n, s)
    
    pri_d = np.dot(psi, pri_x)
    pri_g = np.zeros(m)
    for i in range(m):
        pri_g = pri_d
    
    return pri_g

In [8]:
def CompLaplace(g, deltag, s, K, epsilon):
    m = np.size(g)
    n = int(2**(np.ceil(math.log2(m))))
    k = int(np.floor(s*math.log(n/s)))
    
    t = g[int(s)-1]
    
    x = np.zeros(n)
    for i in range(n):
        if i < m and g[i] >= t:
            x[i] = g[i]
        else:
            x[i] = 0
    
    phi = BernoulliMatrix(k,n)
    psi = HaarMatrix(n)
    A = np.dot(phi, psi)
    
    y = np.dot(A, x)
    
    noise = (2*K/epsilon)*np.sqrt(k)*deltag*((1+(np.sqrt(n)-1)*(np.sqrt(2)+1))/np.sqrt(n))
    pri_y = y + np.random.laplace(loc = 0.0, scale = noise, size = k)
    
    pri_x = OMP(A, pri_y, n, s)
    
    pri_g = g + np.random.laplace(loc = 0.0, scale = 2*K*deltag/epsilon, size=m)
    
    for i in range(m):
        if g[i] >= t:
            pri_g[i] = pri_x[i]
    
    return pri_g

In [9]:
def lap_noise_normal(data, K, epsilon):
    pri_data = data + np.random.laplace(loc = 0.0, scale = 2*K*(4*N/(N+2))/epsilon, size=np.size(data))
    
    return pri_data

In [10]:
def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_mec(stats, s, m, K, epsilon):
    h = np.zeros(m)
    for i in range(m):
        h[i] = stats[i]
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * h[i] / (2*K*s))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        h[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [11]:
K = 3
epsilon = 0.5
eta = 10
s = 10

N = 100
M = 500

stats = GenerateData(N,M)

RT = np.zeros(4)

for i in range(5):
    start = time.time()
    pri_stats = CompressiveMechanism(stats, 4*N/(N+2), K, epsilon, eta)
    end = time.time()
    RT[0] += end-start
    
    print("Comp. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = CompLaplace(stats, 4*N/(N+2), s, K, epsilon)
    end = time.time()
    RT[1] += end-start
    print("Comp. + Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = lap_noise_normal(stats, K, epsilon)
    end = time.time()
    RT[2] += end-start
    print("Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = exp_mec(stats, 4*N/(N+2), M, K, epsilon)
    end = time.time()
    RT[3] += end-start
    print("Exp. = ", end-start, "[sec]")
    print("------------")

print("------------")
print("Average")
RT /= 5
print("Comp. = ", RT[0], "[sec]")
print("Comp. + Lap. = ", RT[1], "[sec]")
print("Lap. = ", RT[2], "[sec]")
print("Exp. = ", RT[3], "[sec]")

Comp. =  6.461298942565918 [sec]
Comp. + Lap. =  3.2047972679138184 [sec]
Lap. =  0.00039696693420410156 [sec]
Exp. =  0.001990079879760742 [sec]
------------
Comp. =  6.293234825134277 [sec]
Comp. + Lap. =  3.145340919494629 [sec]
Lap. =  0.00043702125549316406 [sec]
Exp. =  0.0019102096557617188 [sec]
------------
Comp. =  6.345113277435303 [sec]
Comp. + Lap. =  3.1824839115142822 [sec]
Lap. =  0.00043582916259765625 [sec]
Exp. =  0.0019559860229492188 [sec]
------------
Comp. =  6.2714927196502686 [sec]
Comp. + Lap. =  3.0815279483795166 [sec]
Lap. =  0.0003972053527832031 [sec]
Exp. =  0.002117156982421875 [sec]
------------
Comp. =  6.151737213134766 [sec]
Comp. + Lap. =  3.1323049068450928 [sec]
Lap. =  0.000392913818359375 [sec]
Exp. =  0.0019540786743164062 [sec]
------------
------------
Average
Comp. =  6.304575395584107 [sec]
Comp. + Lap. =  3.149290990829468 [sec]
Lap. =  0.0004119873046875 [sec]
Exp. =  0.001985502243041992 [sec]


In [12]:
K = 3
epsilon = 0.5
eta = 10
s = 10

N = 150
M = 1000

stats = GenerateData(N,M)

RT = np.zeros(4)

for i in range(5):
    start = time.time()
    pri_stats = CompressiveMechanism(stats, 4*N/(N+2), K, epsilon, eta)
    end = time.time()
    RT[0] += end-start
    
    print("Comp. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = CompLaplace(stats, 4*N/(N+2), s, K, epsilon)
    end = time.time()
    RT[1] += end-start
    print("Comp. + Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = lap_noise_normal(stats, K, epsilon)
    end = time.time()
    RT[2] += end-start
    print("Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = exp_mec(stats, 4*N/(N+2), M, K, epsilon)
    end = time.time()
    RT[3] += end-start
    print("Exp. = ", end-start, "[sec]")
    print("------------")

print("------------")
print("Average")
RT /= 5
print("Comp. = ", RT[0], "[sec]")
print("Comp. + Lap. = ", RT[1], "[sec]")
print("Lap. = ", RT[2], "[sec]")
print("Exp. = ", RT[3], "[sec]")

Comp. =  28.235933780670166 [sec]
Comp. + Lap. =  10.878007173538208 [sec]
Lap. =  0.00048422813415527344 [sec]
Exp. =  0.003998994827270508 [sec]
------------
Comp. =  28.338014125823975 [sec]
Comp. + Lap. =  10.935129880905151 [sec]
Lap. =  0.0004971027374267578 [sec]
Exp. =  0.0038750171661376953 [sec]
------------
Comp. =  28.309298992156982 [sec]
Comp. + Lap. =  10.76988697052002 [sec]
Lap. =  0.0004780292510986328 [sec]
Exp. =  0.003942012786865234 [sec]
------------
Comp. =  28.087568283081055 [sec]
Comp. + Lap. =  10.73656177520752 [sec]
Lap. =  0.0004899501800537109 [sec]
Exp. =  0.004069089889526367 [sec]
------------
Comp. =  28.175498962402344 [sec]
Comp. + Lap. =  10.770728349685669 [sec]
Lap. =  0.0004711151123046875 [sec]
Exp. =  0.004011631011962891 [sec]
------------
------------
Average
Comp. =  28.229262828826904 [sec]
Comp. + Lap. =  10.818062829971314 [sec]
Lap. =  0.0004840850830078125 [sec]
Exp. =  0.003979349136352539 [sec]


In [13]:
K = 3
epsilon = 0.5
eta = 10
s = 10

N = 300
M = 2000

stats = GenerateData(N,M)

RT = np.zeros(4)

for i in range(5):
    start = time.time()
    pri_stats = CompressiveMechanism(stats, 4*N/(N+2), K, epsilon, eta)
    end = time.time()
    RT[0] += end-start
    
    print("Comp. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = CompLaplace(stats, 4*N/(N+2), s, K, epsilon)
    end = time.time()
    RT[1] += end-start
    print("Comp. + Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = lap_noise_normal(stats, K, epsilon)
    end = time.time()
    RT[2] += end-start
    print("Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = exp_mec(stats, 4*N/(N+2), M, K, epsilon)
    end = time.time()
    RT[3] += end-start
    print("Exp. = ", end-start, "[sec]")
    print("------------")

print("------------")
print("Average")
RT /= 5
print("Comp. = ", RT[0], "[sec]")
print("Comp. + Lap. = ", RT[1], "[sec]")
print("Lap. = ", RT[2], "[sec]")
print("Exp. = ", RT[3], "[sec]")

Comp. =  165.7092990875244 [sec]
Comp. + Lap. =  39.62262463569641 [sec]
Lap. =  0.0006110668182373047 [sec]
Exp. =  0.00779414176940918 [sec]
------------
Comp. =  166.47248101234436 [sec]
Comp. + Lap. =  40.036765813827515 [sec]
Lap. =  0.0006320476531982422 [sec]
Exp. =  0.00787210464477539 [sec]
------------
Comp. =  166.46508383750916 [sec]
Comp. + Lap. =  40.346495151519775 [sec]
Lap. =  0.0006208419799804688 [sec]
Exp. =  0.0076067447662353516 [sec]
------------
Comp. =  167.4835410118103 [sec]
Comp. + Lap. =  39.58103609085083 [sec]
Lap. =  0.0006351470947265625 [sec]
Exp. =  0.00742793083190918 [sec]
------------
Comp. =  166.42043209075928 [sec]
Comp. + Lap. =  39.794262170791626 [sec]
Lap. =  0.0006449222564697266 [sec]
Exp. =  0.007827281951904297 [sec]
------------
------------
Average
Comp. =  166.5101674079895 [sec]
Comp. + Lap. =  39.87623677253723 [sec]
Lap. =  0.000628805160522461 [sec]
Exp. =  0.0077056407928466795 [sec]


In [14]:
K = 3
epsilon = 0.5
eta = 10
s = 10

N = 500
M = 5000

stats = GenerateData(N,M)

RT = np.zeros(4)

for i in range(5):
    start = time.time()
    pri_stats = CompressiveMechanism(stats, 4*N/(N+2), K, epsilon, eta)
    end = time.time()
    RT[0] += end-start
    
    print("Comp. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = CompLaplace(stats, 4*N/(N+2), s, K, epsilon)
    end = time.time()
    RT[1] += end-start
    print("Comp. + Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = lap_noise_normal(stats, K, epsilon)
    end = time.time()
    RT[2] += end-start
    print("Lap. = ", end-start, "[sec]")
    
    start = time.time()
    pri_stats = exp_mec(stats, 4*N/(N+2), M, K, epsilon)
    end = time.time()
    RT[3] += end-start
    print("Exp. = ", end-start, "[sec]")
    print("------------")

print("------------")
print("Average")
RT /= 5
print("Comp. = ", RT[0], "[sec]")
print("Comp. + Lap. = ", RT[1], "[sec]")
print("Lap. = ", RT[2], "[sec]")
print("Exp. = ", RT[3], "[sec]")

Comp. =  9278.864812135696 [sec]
Comp. + Lap. =  596.5563278198242 [sec]
Lap. =  0.0010921955108642578 [sec]
Exp. =  0.019820213317871094 [sec]
------------
Comp. =  9239.82158088684 [sec]
Comp. + Lap. =  589.2892339229584 [sec]
Lap. =  0.001077890396118164 [sec]
Exp. =  0.02105879783630371 [sec]
------------
Comp. =  9229.745341300964 [sec]
Comp. + Lap. =  585.8827221393585 [sec]
Lap. =  0.0010848045349121094 [sec]
Exp. =  0.01961517333984375 [sec]
------------
Comp. =  9232.183667898178 [sec]
Comp. + Lap. =  584.9788501262665 [sec]
Lap. =  0.001092672348022461 [sec]
Exp. =  0.019539356231689453 [sec]
------------
Comp. =  9219.473161935806 [sec]
Comp. + Lap. =  584.4801259040833 [sec]
Lap. =  0.0010552406311035156 [sec]
Exp. =  0.019643783569335938 [sec]
------------
------------
Average
Comp. =  9240.017712831497 [sec]
Comp. + Lap. =  588.2374519824982 [sec]
Lap. =  0.0010805606842041015 [sec]
Exp. =  0.019935464859008788 [sec]
